In [ ]:
model = SentenceTransformer("dangvantuan/sentence-camembert-large")

In [1]:
pip install transformers

In [2]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for w2v
import gensim
import gensim.downloader as gensim_api
## for bert
import transformers
import pandas as pd
import tarfile
import os
import gc
import random

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows',5000)
from tqdm.autonotebook import tqdm
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC

# Import Libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

C:\Users\hp\AppData\Local\Temp\ipykernel_25028\1840354344.py:30: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
import pandas as pd

# Charger les données
df = pd.read_csv("C:/Users/hp/Downloads/categorized_data (1).csv (2)/categorized_data (1).csv", encoding='utf-8-sig', delimiter=';', header=0)

df= df.drop(['Processed Review','Category1','Category2','Category3','Sentiment','year'], axis=1)

# data = data.sample(frac=1, random_state=42).reset_index(drop=True)
df.head(5)

,Date,Review,Rating
0,05/05/2023,"A FUIR !!!!!!!!!!!!!!! Très mécontente, après avoir attendu 6 heures le technicien (dans un appartement loin de chez moi où je ne pouvais rien faire, et ayant dû poser ma journée) personne n'est venu, on m'a promis un dédommagement au téléphone et maintenant ils disent qu'il n'y a pas d'écrit et que cela fait plus de deux mois (j'ai relancé entre temps 2 fois !).... ULTRA mécontente, une vraie ARNAQUE !",1
1,05/05/2023,"Prise de rdv pour une installation fibre chez ma grand-mère. Le technicien refuse d'acheminer le câble où elle lui demande en prétextant des impossibilités techniques, n'en fait qu'à sa tête et creuse un trou dans la terrasse pour faire passer le câble. Ma mère est obligée d'intervenir et lui demande de tout arrêter. Il s'en va. Nouvelle prise de rendez-vous, nous demandons à changer de sous-traitant. Le rdv est pris pour un autre matin. Puis ce matin là à 9h, il est déplacé pour 13h et finalement le technicien ne viendra pas. Le rendez-vous est alors reporté au lendemain après-midi et une conseillère me planifie un rdv téléphonique à la suite de ce rdv (elle ne l'a bien sûr pas honoré). Finalement c'est le même sous-traitant, mais un technicien plus expérimenté. Il rebouche le trou (bien sur pas la bonne couleur). L'installation se passe bien. 1 journée et demie de perdu (= congés sans soldes), 1 terrasse détériorée. Je demande un dédommagement pour tout cela. On me propose un mois gratuit. Soit 10% de la somme que ça a engendré en frais annexe. Rien de plus, à peine des excuses, la conseillère absolument pas empathique ou sympathique, même pas de bonne journée en fin d'appel. Évidemment pour arriver à avoir cette personne je suis passée par au moins 10 interlocuteurs. Je suis dégoutée par ce traitement. Je suis dégoutée par SFR qui impose un certains nombre d'interventions à ses sous-traitants et par conséquent qu'ils ne peuvent pas honorés leurs rendez-vous. Je suis dégoutée du mépris du 1er technicien envers ma grand-mère. Pour l'instant le téléphone ne fonctionne toujours pas car le câble tel est différent entre l'ADSL et la fibre mais ça ce n'est rien par rapport au reste...",1
2,05/02/2023,"Nous n'arrivions pas à faire changer notre connect tv auprès du service client qui disfonctionnait. Au bout de 3 mois nous avons réussi et ils ont envoyé le matériel à notre ancienne adresse...! Après ça, les problèmes de la Connect tv étaient les mêmes... nous avons dû acheter notre propre matériel.",1
3,05/01/2023,Je suis partie vivre à l’étranger. J’ai donc retiré ma carte SIM de mon téléphone. J’ai reçu des factures de téléphone avec du hors forfait. Je souligne bien que ma puce n’était même pas dans mon téléphone. À boycotter SFR ! Des voleurs !,1
4,21/04/2023,"Zéro étoile j’ai résilié une option netflix depuis 20 février 2022 et on me prélève toujours on est le 21/04/2023 sois disant sans engament. Oui je suis d’accord avec les autre avis des gros voleurs et leur service clients déplorables qui touche un salaire et vous font croire qui vont resoudre le problème, il sont entrain de couler l’entreprise, fuyez pauvre fou.",1


In [13]:
import re
import string
import nltk
from nltk.corpus import stopwords

def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()

    # Supprimer les caractères spéciaux et les emojis
#     text = emoji.demojize(text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r':[a-z_]+:', ' ', text)

    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenisation
    tokens = text.split()

    # Supprimer les stopwords
    stop_words = set(stopwords.words('french'))
    tokens = [token for token in tokens if token not in stop_words]

#     # Lemmatisation avec spaCy
#     doc = nlp(text)
#     tokens = [token.lemma_ for token in doc]

#     # Stemming
#     stemmer = SnowballStemmer("french")
#     tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)

In [14]:
df['Review_cleaned'] = df['Review'].apply(preprocess_text)

In [28]:
df= df.head(300)

In [29]:
df.shape

(300, 5)

In [16]:
#### Préparer les phrases pour chaque catégorie
categories = {
    'Résiliation': [
        "Quitté pour un autre opérateur avec une portabilité le 11 janvier.",
         "déjà j'avais quitté red mobile",
         "J’ai essayé résilier",
         "Je me suis retrouvée avec mon contrat résilié.",
          "Lorsque je demande une résiliation.",
         "J'ai demandé la résiliation de ma box",
         "Le client exprime son souhait de résilier son offre et mentionne que l'opérateur demande des frais de résiliation.",
         "Alors pour la résiliation bordel .",
         "J'ai demandé la résiliation de ma box Internet le 14 juin.",
        "lorsque vous résiliez votre contrat, n'applique pas la résiliation",
        "La résiliation sera automatique de ce fait.",
        " Demandes de résiliation par formulaire",
        " j'ai voulu resilier.",
        "RESILIATION.",
        " apres resilation de red by sfr"
        ],
    
    'Image de marque': [

        " Des voleurs et des menteurs. Impossibilité de les joindre sauf par messenger. Attention arnaque",
        "red SFR box à fuir.",
        "Je pars donc de chez eux car j'en peux plus, c'est le coup de grâce !! Se sont des voleurs !!! A fuir !!!",
        "Quitté pour un autre opérateur",
        "Fuyez ou changez.",
        "A fuir, je ne recommande pas, on se sent volé!",
        "Je pense changer d’opérateur.",
        "Fuyez ou changez."

                  ],
    
    'Service Client': [ 
    
            "Impossible de joindre un conseillé N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux."
            ,"Le client a contacté plusieurs services clients, mais n'a obtenu aucune réponse satisfaisante. ."
            ,"Contact par chat uniquement avec des conseillers incompétents qui ne savent pas pourquoi la demande est rejetée, et qui vous renvoient au site ou à faire un courrier à SFR. Le service client",
            "Service client deplorable.. Elle a fini par mettre fin à la conversation sans attendre réponse de ma part.",
            "ont peut que communiquer avec eux que par message SMS",
            "Lorsque je les ai contacté",
            "J'appelle ou je tchat quotidiennement avec le service clientèle qui n'est toujours pas en mesure de me donner un rdv pour installer et raccorder ma ligne.",
            "Impossible de parler à un humain de vive voix ou de face à face.",
            "Et impossible après 29 tentatives de les joindre pour remboursement de la période du 11 au 30. Drôle de manières !!!",
            "c'est inacceptable", "je choisis les offres qui sont respectées a vie", "et red ne fait plus partie de ces revendeurs.",
            "j'ai fait 6 demandes et de multiples appels et chat en ligne et la demande n'aboutit pas. Le SAV est totalement déficient !!!",
            "Service client incompétent ",
            "un coup de fil, et généralement le problème est résolu rapidement",
            "Service client d’une incompétence… à ce niveau là c’est du jamais vu. Une fois sur 2 on vous raccroche au nez.",
            "Service client incompetent",
            "Au bout de plus d’un mois à joindre leurs services quotidiennement.",
            "Le service consommateur lit un script et ne sait pas quoi dire si on sort du script.",
            "Aucun vrai conseiller, seulement un Bot. ça fait deux heures que j'essaie de résoudre un problème, et toutes les deux minutes ça se réinitialise et me repose les mêmes questions.",
            "SAV compétent et professionnel",
            "des dizaines d'échanges avec le service client, sur l'application et par téléphone",
            "Le service client inactif ",


],
    
    'Forfait/Offre': [ 

        "On vous promet une offre à vie et hop d un couo de baguette magique on vous augmente de 30%",
        "Pub TV et sans prévenir",
        "Je vous propose de profitez de 15€ offerts lors de votre souscription d'abonnement chez Red. (15€ pour Red Box et 10€ pour Red mobile)",
        "quand on sait que a la poste le forfait est a 9.99 euro pour 50 go dont 15 go internationale et A VIE.",
         "Et lorsque vous avez quelqu'un, il vous promet un geste commercial que vous n'avez jamais, simplement pour se débarrasser de vous!",
        " Netflix mais au final ils ne m'offrent ni Netflix ni le bouquet, super de faire perdre du temps aux gens pour rien.",
        "Pour mes deux offres, je me retrouve régulièrement avec des options pour lesquelles je n'ai fais aucunes demandes",
        "Il vous change votre abonnement sans vous prévenir pour une meilleure offre sois disant avec une prime en plus de 3 €",
        "Je prends un abonnement + achat d'un téléphone pour SFR RED 200 GO; la ligne est opérationnelle le 18/03/23 après réception du matériel et carte Sim. Tout fonctionne bien."
        " j'ai un forfait mobile soit chez SFR et par la suite red",
        ],
    
    
        'Facturation':     [

                "mon problème de facturation a été géré"
                "Je n'ai jamais rencontré de problèmes avec la facturation ",
                "Me facture plus cher",
                "Les factures reçues sont supérieures à ce qui a été convenu",
                "Après plusieurs minutes à essayer d'avoir un details de facture la facture est correcte.",
                "Le nombre de fois que je me suis retrouvée avec des factures augmentées on ne sait pas pourquoi... Ils vous rajoutent des options sans que vous ayez rien demandé !",
                "Ils m'ont changé mon forfait sans aucune demande de ma part (toujours pareil sans aucun consentement de ma part), bien évidemment beaucoup plus cher !!!!",
                "Quitté pour un autre opérateur avec une portabilité le 11 janvier, ils retirent sur ton compte l'intégralité forfait de janvier et des portions telles que Deezer.",
                "Leur prix augmentent sans notification de leur part de quelques euros chaque année.",
                "tous les mois paye et non remboursé pour une box dans un carton.",
                "surfacturation mensuelle d'une option tv que je n ai pas",
                "Prix en perpetuelle augmentation",
                "Elle n'a pas été effective et j'ai été facturée les mois de juin, juillet et août. À ce jour, donc en octobre, j'attends toujours le remboursement après des dizaines d'échanges avec le service client, sur l'application et par téléphone, qui me confirment à chaque fois que je vais être remboursée mais ce est absolument pas le cas. attends toujours.",
                "J'ai été facturée les mois de juin, juillet et août.",
                "Je me retrouve donc certains mois avec +3€ par-ci, +5€ par là.",
                "Je suis très satisfait de la façon dont mon fournisseur de téléphonie traite les problèmes de facturation.",
                "Je suis très mécontent de la facturation exorbitante",



                ],
    
    


                'Debit internet' :  [

            "La vitesse de l'internet est vraiment rapide et stable!"
            "Problème sur ma fibre optique",
            "pas moyen de passer un coup de fil sans coupure, ils m'ont dit configurer en appels wifi pour éviter le probleme, ok mais dans ce cas à quoi cela sert il que je paye un abonnement chez eux. minable",
            " sans internet alors que c'est le seul outil nécessaire.",
            "ces 4g box sont les plus éclatées que j'ai jamais eu de ma vie",
            "même sans ces bugs la connexion n'est pas stable",
            "Problème sur ma ligne fixe FIBRE apparu le 1 octobre",
            "Problème sur ma fibre optique",
            "la fibre plante un jour sur deux.",
            "une offre fibre",
            "pres 6 mois de fibre chez Red by Sfr",
            "Problèmes de débit"
            ],

            'Installation/Service technique' : [


            " un technicien est programmé pour venir le lendemain 09/03 entre 10h et 12h. Évidemment personne ne vient....",
            "un rdv pour installer et raccorder ma ligne.",
             "2 semaines d'attentes pour avoir un technicien.",
            " aucune trace du technicien le jour J.",
            " technicien absent, problème toujours présent.",
            "Et les techniciens sont tous en sous-traitances ce qui fait qu'ils ne sont pas investis.",
             "Le technicien ne vient pas au rdv.",
             "Des rendez-vous non honorés de la part des techniciens, un problème qui vient toujours d’ailleurs.",
            "Problèmes techniques récurrents avec la fibre. Ça fait 3 mois que nous l'utilisons, nous en sommes à notre 5ème dépannage pour coupure l'Internet (dont 2 la même semaine).",
            "30 minutes d'attente technicien qui ne se présente pas et pas de disponibilité pour un dépannage de box qui coupe sans arrêt",
            "technicien qui ne se présente pas",
            "il faut appeler un technicien, qui la plupart du temps met plus d'une semaine à arriver et ne sert à rien",


                ],


            'Contrat' :  [
            "Le contrat téléphonique est facile à gérer"
            " changement de titulaire sur une ligne existante, ."   ,
            "Tiens tiens tiens ... 2 mois.... c'est justement le délai légal dans lequel le FAI doit impérativement avoir effectué le raccordement. Au delà il est susceptible d'avoir des problèmes juridiques et comme par hasard on me propose ce jour là d'annuler et recommencer la procédure ...",
             "ont tendance a vouloir imposer des options",
             "On fait des erreurs sur votre contrat.",
            "la fidélité ne paye pas même pour un service sans engagement.",
            "lors de ma souscription j'étais sous liste rouge,",
            " sans pour autant changer le contrat.",
            "rien de cela n'est stipulé dans mon contrat",
            "J'ai demandé la résiliation de mon contrat.",
            "On fait des erreurs sur votre contrat… incroyable!"


            ],


            'Couverture réseau' : ["de la couverture réseau de l'opérateur.",
                                 "Le réseau est très lent dans ma région et j'ai souvent des problèmes de connexion.",
                                 "La qualité du réseau est très variable selon l'endroit où je me trouve.",
                                 "Le service réseau est très cher par rapport à la qualité de la connexion.",
                                 "J'ai rencontré des problèmes pour passer des appels et envoyer des SMS dans ma région.",
                                 "Le réseau de l'opérateur est très fiable et n'a jamais été interrompu.",
                                 "Le débit Internet est très instable et change souvent.",
                                 "Je suis souvent coupé lors de mes appels téléphoniques en raison d'un signal faible.",
                                 "Le réseau est souvent en panne et le service technique ne semble pas capable de le réparer.",
                                 "La couverture réseau de l'opérateur est très limitée par rapport à la concurrence."
                                 " le réseau SFR nul ",
                                 "Problèmes de coupures intempestives du réseau",

                                ]

}

In [1]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [18]:
from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer("dangvantuan/sentence-camembert-large", device='cpu')
model = SentenceTransformer("dangvantuan/sentence-camembert-large")

No sentence-transformers model found with name C:\Users\hp/.cache\torch\sentence_transformers\dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


In [19]:
# Encoder les phrases de chaque catégorie
embeddings = {}
for category, sentences in categories.items():
    embeddings[category] = model.encode(sentences, convert_to_tensor=True,batch_size=1)

In [20]:
embeddings[category]

tensor([[ 0.3661,  0.1513,  0.3438,  ..., -0.1320,  0.0182, -0.2364],
        [ 0.4657,  0.2827,  0.3987,  ...,  0.1490,  0.0832, -0.2711],
        [ 0.4073,  0.2155, -0.0090,  ..., -0.0492,  0.1331,  0.0577],
        ...,
        [-0.0428, -0.1424, -0.1096,  ...,  0.0560, -0.1593, -0.3396],
        [ 0.2391,  0.1677,  0.2172,  ..., -0.1963,  0.0582, -0.0734],
        [-0.0073,  0.1699,  0.4972,  ...,  0.0868,  0.0136, -0.3541]])

In [21]:
embeddings[category].shape

torch.Size([11, 1024])

In [22]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [25]:
max_length=1024
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("camembert-base")



for index, row in df.iterrows():
    review = row['Review_cleaned']

    # Tokenisation et limitation de la longueur
    tokens = tokenizer.tokenize(review)
    if len(tokens) > max_length:
        tokens = tokens[:max_length]

    review = tokenizer.convert_tokens_to_string(tokens)

In [47]:
max_length=1024
from transformers import CamembertModel, CamembertTokenizer


for index, row in df.iterrows():
    review = row['Review']

    # Tokenisation et limitation de la longueur
    tokens = tokenizer.tokenize(review)
    if len(tokens) > max_length:
        tokens = tokens[:max_length]

    review = tokenizer.convert_tokens_to_string(tokens)

    # Encoder la revue
    try:
        review_embedding = model.encode([review], convert_to_tensor=True)
    except Exception as e:
        print(f"Error at row {index}: {e}")
        continue

In [30]:
from torch.nn.functional import cosine_similarity

for index, row in df.iterrows():
    review = row['Review_cleaned']

    # Tokenisation et limitation de la longueur
    tokens = tokenizer.tokenize(review)
    if len(tokens) > max_length:
        tokens = tokens[:max_length]

    review = tokenizer.convert_tokens_to_string(tokens)

    # Encoder la revue
    try:
        review_embedding = model.encode([review], convert_to_tensor=True)
    except Exception as e:
        print(f"Error at row {index}: {e}")
        continue

    max_similarity = -1  # Initialisez à une valeur très faible
    max_category = None

    # Parcourez chaque catégorie et chaque phrase dans chaque catégorie :
    for category, phrase_embeddings in embeddings.items():
        for phrase_embedding in phrase_embeddings:
            # Calculez la similarité cosinus entre l'embedding de la revue et l'embedding de la phrase :
            similarity = cosine_similarity(review_embedding, phrase_embedding)

            # Si cette similarité est plus grande que la similarité maximale actuelle, mettez à jour la similarité maximale et la catégorie maximale :
            if similarity > max_similarity:
                max_similarity = similarity
                max_category = category

    # À ce stade, max_category est la catégorie de la phrase avec la plus grande similarité à la revue.
    # Vous pouvez maintenant attribuer cette catégorie à la revue dans votre dataframe :
    df.at[index, 'Category'] = max_category

In [32]:
df.head(2)

,Date,Review,Rating,Review_cleaned,Category
0,05/05/2023,"A FUIR !!!!!!!!!!!!!!! Très mécontente, après avoir attendu 6 heures le technicien (dans un appartement loin de chez moi où je ne pouvais rien faire, et ayant dû poser ma journée) personne n'est venu, on m'a promis un dédommagement au téléphone et maintenant ils disent qu'il n'y a pas d'écrit et que cela fait plus de deux mois (j'ai relancé entre temps 2 fois !).... ULTRA mécontente, une vraie ARNAQUE !",1,a fuir très mécontente après avoir attendu 6 heures technicien appartement loin chez où pouvais rien faire dû poser journée personne venu a promis dédommagement téléphone maintenant disent a écrit cela fait plus deux mois relancé entre temps 2 fois ultra mécontente vraie arnaque,Installation/Service technique
1,05/05/2023,"Prise de rdv pour une installation fibre chez ma grand-mère. Le technicien refuse d'acheminer le câble où elle lui demande en prétextant des impossibilités techniques, n'en fait qu'à sa tête et creuse un trou dans la terrasse pour faire passer le câble. Ma mère est obligée d'intervenir et lui demande de tout arrêter. Il s'en va. Nouvelle prise de rendez-vous, nous demandons à changer de sous-traitant. Le rdv est pris pour un autre matin. Puis ce matin là à 9h, il est déplacé pour 13h et finalement le technicien ne viendra pas. Le rendez-vous est alors reporté au lendemain après-midi et une conseillère me planifie un rdv téléphonique à la suite de ce rdv (elle ne l'a bien sûr pas honoré). Finalement c'est le même sous-traitant, mais un technicien plus expérimenté. Il rebouche le trou (bien sur pas la bonne couleur). L'installation se passe bien. 1 journée et demie de perdu (= congés sans soldes), 1 terrasse détériorée. Je demande un dédommagement pour tout cela. On me propose un mois gratuit. Soit 10% de la somme que ça a engendré en frais annexe. Rien de plus, à peine des excuses, la conseillère absolument pas empathique ou sympathique, même pas de bonne journée en fin d'appel. Évidemment pour arriver à avoir cette personne je suis passée par au moins 10 interlocuteurs. Je suis dégoutée par ce traitement. Je suis dégoutée par SFR qui impose un certains nombre d'interventions à ses sous-traitants et par conséquent qu'ils ne peuvent pas honorés leurs rendez-vous. Je suis dégoutée du mépris du 1er technicien envers ma grand-mère. Pour l'instant le téléphone ne fonctionne toujours pas car le câble tel est différent entre l'ADSL et la fibre mais ça ce n'est rien par rapport au reste...",1,prise rdv installation fibre chez grand mère technicien refuse acheminer câble où demande prétextant impossibilités techniques fait tête creuse trou terrasse faire passer câble mère obligée intervenir demande tout arrêter va nouvelle prise rendez demandons changer sous traitant rdv pris autre matin puis matin là 9h déplacé 13h finalement technicien viendra rendez alors reporté lendemain après midi conseillère planifie rdv téléphonique suite rdv a bien sûr honoré finalement sous traitant technicien plus expérimenté rebouche trou bien bonne couleur installation passe bien 1 journée demie perdu congés sans soldes 1 terrasse détériorée demande dédommagement tout cela propose mois gratuit 10 somme ça a engendré frais annexe rien plus peine excuses conseillère absolument empathique sympathique bonne journée fin appel évidemment arriver avoir cette personne passée moins 10 interlocuteurs dégoutée traitement dégoutée sfr impose certains nombre interventions sous traitants conséquent peuvent honorés leurs rendez dégoutée mépris 1er technicien envers grand mère instant téléphone fonctionne toujours car câble tel différent entre adsl fibre ça rien rapport reste,Installation/Service technique
